# Métricas clínicas a partir de segmentaciones vertebrales
En este cuaderno, se calculan las medidas translacionales y angulares sobre vértebras segmentadas.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import datetime
from scipy.stats import t


In [2]:
ROJO = (255, 0, 0)
VERDE = (0, 255, 0)
AZUL = (0, 0, 255)
NARANJA = (255, 165, 0)
AMARILLO = (255, 255, 0)
VIOLETA = (238, 130, 238)
HOY = datetime.datetime.now().strftime("%Y-%m-%d")

In [3]:
ruta_procesado = 'dataset_procesado'

con_mascara = 'dataset_con_mascara'
sin_mascara = 'dataset_sin_mascara'

mod_cervical = 'cervical'
mod_lumbar = 'lumbar'
mod_general = 'general'

imagenes = 'imagenes'
mascaras = 'mascaras'
mascara_limpias = 'segmentaciones_limpias'

metricas = 'metricas_clinicas'
translacionales = 'translacionales'
angulares = 'angulares'


In [4]:
def calcular_centroides(contorno):
  if len(contorno) > 5:
      rectangulo = cv2.minAreaRect(contorno)
      centro = rectangulo[0]
      centroide = (int(centro[0]), int(centro[1]))
  return centroide

## Cálculo de métricas clínicas
### Medidas translacionales
Para cada par consecutivo de vértebras se calcula el desplazamiento lateral, medido como la distancia perpendicular a la recta entre vértebras.  
Estas medidas permiten detectar desviaciones laterales anómalas.

In [ ]:
def obtener_extremos_angulo_vertebra(contorno, centroide, vector_columna):
  puntos = contorno.reshape(-1, 2)

  proyecciones = [np.dot(p - centroide, vector_columna) for p in puntos]
  idx_min = np.argmin(proyecciones)
  idx_max = np.argmax(proyecciones)

  # Obtener el angulo de la vertebra
  rect = cv2.minAreaRect(contorno)
  (cx, cy), (w, h), angulo = rect
  # Posibles angulos
  if w > h:
      angulo_mayor = angulo
      angulo_menor = (angulo + 90) % 180
  else:
      angulo_mayor = (angulo + 90) % 180
      angulo_menor = angulo
  
  eje_mayor = np.array([np.cos(np.radians(angulo_mayor)), np.sin(np.radians(angulo_mayor))])
  eje_menor = np.array([np.cos(np.radians(angulo_menor)), np.sin(np.radians(angulo_menor))])

  v_union = vector_columna / (np.linalg.norm(vector_columna) + 1e-8)
  cos_mayor = np.dot(v_union, eje_mayor)
  cos_menor = np.dot(v_union, eje_menor)

  if abs(cos_mayor) < abs(cos_menor):
    angulo_final = angulo_mayor
    eje_final = eje_mayor
  else:
    angulo_final = angulo_menor
    eje_final = eje_menor

  return puntos[idx_min], puntos[idx_max], angulo_final, eje_final # punto inferior, punto superior, angulo (en grados)

In [6]:
def obtener_punto_derecha_vertebra(contorno, centroide, vector_columna):
  puntos = contorno.reshape(-1, 2)
  vector_perpendicular = np.array([-vector_columna[1], vector_columna[0]])
  if vector_perpendicular[0] < 0:
      vector_perpendicular = -vector_perpendicular
  proyecciones = [np.dot(p - centroide, vector_perpendicular) for p in puntos]
  idx_max = np.argmax(proyecciones)
  return puntos[idx_max] # punto derecho

In [7]:
def proyectar_punto(punto, angulo, eje_principal):
  angulo_rad = np.radians(angulo)
  v1 = np.array([np.cos(angulo_rad), np.sin(angulo_rad)])

  p1, p2 = eje_principal

  v2 = p2 - p1
  A = np.array([v1, -v2]).T 
  b = p1 - punto

  try:
    t_s = np.linalg.solve(A, b)
    punto_proyectado = punto + t_s[0] * v1
    return punto_proyectado
  except np.linalg.LinAlgError:
    print("Error: No se puede resolver el sistema de ecuaciones.")
    return None

In [8]:
def normalizar_angulo(angulo):
  angulo = (angulo % 180) % 360 - 180
  if angulo > 90:
    return angulo - 180
  elif angulo < -90:
    return angulo + 180
  return angulo

In [9]:
def promedio_angular(angulo1, angulo2):
  # si el signo de los angulos es distinto
  if (angulo1 > 0 and angulo2 < 0) or (angulo1 < 0 and angulo2 > 0):
    if angulo1 < 0:
      angulo1 = -angulo1
    if angulo2 < 0:
      angulo2 = -angulo2
  v1 = np.array([np.cos(np.radians(angulo1)), np.sin(np.radians(angulo1))])
  v2 = np.array([np.cos(np.radians(angulo2)), np.sin(np.radians(angulo2))])
  v_avg = (v1 + v2) / 2
  norm = np.linalg.norm(v_avg)
  if norm < 1e-8:
    v_avg = np.array([np.cos(np.radians(angulo1)), np.sin(np.radians(angulo1))])
  else:
    v_avg = v_avg / norm

  angulo_promedio = np.degrees(np.arctan2(v_avg[1], v_avg[0]))
  return angulo_promedio

In [ ]:
def calcular_medidas_translacionales(mascara, imagen, nombre, debug=False, tipo=None):
  # hacer copia de la imagen para pintar los resultados
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
  imagen_debug = imagen.copy()

  contornos, _ = cv2.findContours(mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contornos = sorted(contornos, key=lambda c: cv2.boundingRect(c)[1])

  distancias = []
  ## Para cada par de vertebras, calcular el ángulo entre los vectores que las unen
  for i in range(len(contornos)-1):
    j = i + 1
    contorno1 = contornos[i]
    contorno2 = contornos[j]

    cv2.drawContours(imagen, contornos, i, AMARILLO, 2)
    cv2.drawContours(imagen, contornos, j, AMARILLO, 2)
    
    centroide1 = calcular_centroides(contorno1)
    centroide2 = calcular_centroides(contorno2)
    vector = np.array([centroide2[0] - centroide1[0], centroide2[1] - centroide1[1]])
    vector = vector / (np.linalg.norm(vector) + 1e-8)

    _, punto_sup, angulo_sup, eje_sup = obtener_extremos_angulo_vertebra(contorno1, centroide1, vector) # borde superior de vertebra inferior
    punto_inf, _, angulo_inf, eje_inf = obtener_extremos_angulo_vertebra(contorno2, centroide2, vector) # borde inferior de vertebra superior

    angulo_sup = normalizar_angulo(angulo_sup)
    angulo_inf = normalizar_angulo(angulo_inf)

    p1 = centroide1 + eje_sup * 100
    p2 = centroide1 - eje_sup * 100
    cv2.line(imagen_debug, tuple(p1.astype(int)), tuple(p2.astype(int)), VIOLETA, 2)
    cv2.putText(imagen_debug, f"{angulo_sup:.2f}", tuple(centroide1), cv2.FONT_HERSHEY_SIMPLEX, 2, AZUL, 2)
    p1 = centroide2 + eje_inf * 100
    p2 = centroide2 - eje_inf * 100
    cv2.line(imagen_debug, tuple(p1.astype(int)), tuple(p2.astype(int)), VIOLETA, 2)   
    cv2.line(imagen_debug, centroide1, centroide2, VERDE, 2)
    cv2.putText(imagen_debug, f"{angulo_inf:.2f}", tuple(centroide2), cv2.FONT_HERSHEY_SIMPLEX, 2, AZUL, 2)

    
    punto_medio = (punto_sup + punto_inf) / 2
    angulo_medio = promedio_angular(angulo_sup, angulo_inf)
    angulo_rad = np.radians(angulo_medio)
    vector_director = np.array([np.cos(angulo_rad), np.sin(angulo_rad)])

    cv2.circle(imagen_debug, tuple(punto_medio.astype(int)), 10, ROJO, 3)

    largo = 200
    p1 = punto_medio + vector_director * largo
    p2 = punto_medio - vector_director * largo
    eje_intermedio = [p1, p2]
    cv2.line(imagen, tuple(eje_intermedio[0].astype(int)), tuple(eje_intermedio[1].astype(int)), NARANJA, 2)
    cv2.line(imagen_debug, tuple(eje_intermedio[0].astype(int)), tuple(eje_intermedio[1].astype(int)), NARANJA, 4)

    # Proyecciones
    # Proyección del punto superior
    angulo_sup_perpendicular = (angulo_sup + 90) % 180
    punto_sup_derecha = obtener_punto_derecha_vertebra(contorno1, centroide1, vector)
    proyectado_sup = proyectar_punto(punto_sup_derecha, angulo_sup_perpendicular, eje_intermedio)
    largo = 40
    angulo_rad = np.radians(angulo_sup_perpendicular)
    vector_proy_sup = np.array([np.cos(angulo_rad), np.sin(angulo_rad)])
    p1 = proyectado_sup + vector_proy_sup * largo
    p2 = proyectado_sup - vector_proy_sup * largo
    cv2.line(imagen, tuple(p1.astype(int)), tuple(p2.astype(int)), ROJO, 2)
    cv2.line(imagen_debug, tuple(p1.astype(int)), tuple(p2.astype(int)), ROJO, 2)
    # Proyección del punto inferior
    angulo_inf_perpendicular = (angulo_inf + 90) % 180
    punto_inf_derecha = obtener_punto_derecha_vertebra(contorno2, centroide2, vector)
    proyectado_inf = proyectar_punto(punto_inf_derecha, angulo_inf_perpendicular, eje_intermedio)
    largo = 40
    angulo_rad = np.radians(angulo_inf_perpendicular)
    vector_proy_inf = np.array([np.cos(angulo_rad), np.sin(angulo_rad)])
    p1 = proyectado_inf + vector_proy_inf * largo
    p2 = proyectado_inf - vector_proy_inf * largo
    cv2.line(imagen, tuple(p1.astype(int)), tuple(p2.astype(int)), ROJO, 2)
    cv2.line(imagen_debug, tuple(p1.astype(int)), tuple(p2.astype(int)), ROJO, 2)
    if proyectado_sup is not None and proyectado_inf is not None:
      distancia = np.linalg.norm(proyectado_sup - proyectado_inf)
      distancias.append(distancia)

      cv2.line(imagen, tuple(proyectado_sup.astype(int)), tuple(proyectado_inf.astype(int)), VERDE, 2)
      cv2.line(imagen_debug, tuple(proyectado_sup.astype(int)), tuple(proyectado_inf.astype(int)), AZUL, 2)

      punto_texto = ((proyectado_sup + proyectado_inf) / 2).astype(int)
      letra_tam = 3
      if 'lumbar' in nombre.lower():
        letra_tam = 3
      cv2.putText(imagen, f"{distancia:.2f}", tuple(punto_texto), cv2.FONT_HERSHEY_SIMPLEX, letra_tam, VERDE, 5)

  if debug:
    for i, c in enumerate(contornos):
      rect = cv2.minAreaRect(c)
      box = cv2.boxPoints(rect)
      box = np.int0(box)
      angulo = rect[2]
      cv2.drawContours(imagen_debug, [box], 0, AMARILLO, 2)
      centro = (int(rect[0][0]), int(rect[0][1]))
      #cv2.putText(imagen_debug, f"{angulo:.2f}", centro, cv2.FONT_HERSHEY_SIMPLEX, 1, AZUL, 2) # angulo de la vertebra
    print("Diferencias translacionales:", distancias)
    angulos=[]

    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.title("Imagen original")
    plt.imshow(imagen)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.title("Imagen con medidas")
    plt.imshow(imagen_debug)
    plt.axis('off')
    
    # guardar la figura 
    ruta_guardar = os.path.join('resultados_metricas', tipo, 'translacionales', nombre)
    os.makedirs(os.path.dirname(ruta_guardar), exist_ok=True)
    plt.savefig(ruta_guardar, bbox_inches='tight')
    # devolver la imagen con todo pintada y una lista de los ángulos
    plt.show()
  
  return imagen, distancias

### Medidas angulares
Se calculan los ángulos formados por pares consecutivos de vértebras, tomando los vectores entre centroides.  
Esta métrica es útil para detectar curvaturas anómalas.

In [11]:
def eje_principal_vertebra(contorno, vector, imagen, debug=False):
  rectangulo = cv2.minAreaRect(contorno)
  (cx, cy), (w, h), angulo = rectangulo

  if debug:
    box = cv2.boxPoints(rectangulo)
    box = np.int0(box)
    cv2.drawContours(imagen, [box], 0, ROJO, 2)

  if w > h:
    angulo_mayor = angulo
    angulo_menor = (angulo + 90) % 180
  else:
    angulo_mayor = (angulo + 90) % 180
    angulo_menor = angulo

  eje_mayor = np.array([np.cos(np.radians(angulo_mayor)), np.sin(np.radians(angulo_mayor))])
  eje_menor = np.array([np.cos(np.radians(angulo_menor)), np.sin(np.radians(angulo_menor))])

  v_union = vector / (np.linalg.norm(vector) + 1e-8)
  cos_mayor = np.dot(v_union, eje_mayor)
  cos_menor = np.dot(v_union, eje_menor)

  if abs(cos_mayor) < abs(cos_menor):
    eje = eje_mayor
    angulo = angulo_mayor
  else:
    eje = eje_menor
    angulo = angulo_menor
  
  return eje, angulo

In [12]:
def diferencia_angulo(vector1, vector2):
  v1_u = vector1 / (np.linalg.norm(vector1) + 1e-8)
  v2_u = vector2 / (np.linalg.norm(vector2) + 1e-8)
  dot = np.clip(np.dot(v1_u, v2_u), -1, 1) # producto escalar entre los dos vectores unitarios (da el angulo entre ellos)
  return np.degrees(np.arccos((np.abs(dot))))

In [ ]:
def calcular_medidas_angulares(mascara, imagen, nombre, debug=False, tipo=None):
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
  imagendebug = imagen.copy()
  contornos, _ = cv2.findContours(mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  contornos = sorted(contornos, key=lambda c: cv2.boundingRect(c)[1])
  # Para cada par de vertebras, calcular el ángulo entre los vectores que las unen
  ejes_principales = []
  centroides = []
  for i in range(len(contornos)-1):
    j = i + 1
    contorno1 = contornos[i]
    contorno2 = contornos[j]
    cv2.drawContours(imagen, contornos, i, AMARILLO, 2)
    cv2.drawContours(imagen, contornos, j, AMARILLO, 2)
    
    centroide1 = calcular_centroides(contorno1)
    centroide2 = calcular_centroides(contorno2)
    vector = (centroide2[0] - centroide1[0], centroide2[1] - centroide1[1])

    if i == 0:
      eje_principal1, angulo1 = eje_principal_vertebra(contornos[i], vector, imagendebug, debug=True)
      ejes_principales = [eje_principal1]
      centroides = [centroide1]
    else:
      eje_principal1 = ejes_principales[-1]
    
    eje_principal2, angulo2 = eje_principal_vertebra(contornos[j], vector, imagendebug, debug=True)
    ejes_principales.append(eje_principal2)
    centroides.append(centroide2)

  for centroide, eje in zip(centroides, ejes_principales):
    extremo1 = (int(centroide[0] + eje[0] * 200), int(centroide[1] + eje[1] * 200))
    extremo2 = (int(centroide[0] - eje[0] * 200), int(centroide[1] - eje[1] * 200))
    cv2.line(imagen, extremo1, extremo2, ROJO, 2)
    cv2.line(imagendebug, extremo1, extremo2, ROJO, 2)

  diferencias = []
  for i in range(len(ejes_principales)-1):
    angulo = diferencia_angulo(ejes_principales[i], ejes_principales[i+1])
    diferencias.append(angulo)

    # Pintar el ángulo entre los ejes principales
    c1 = centroides[i]
    c2 = centroides[i+1]
    cx = int((c1[0] + c2[0]) / 2)
    cy = int((c1[1] + c2[1]) / 2)
    letra_tam = 3
    if 'lumbar' in nombre.lower():
      letra_tam = 3
    cv2.putText(imagen, f"{angulo:.3f}", (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, letra_tam, VERDE, 5)
  
  if debug:
    print('Ejes principales:', ejes_principales)
    print('Diferencias:', diferencias)
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.title('Imagen original')
    plt.imshow(imagen)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.title('Imagen con cajas')
    plt.imshow(imagendebug)
    plt.axis('off')
    
    # guardar la figura 
    ruta_guardar = os.path.join('resultados_metricas', tipo, 'angulares', nombre)
    os.makedirs(os.path.dirname(ruta_guardar), exist_ok=True)
    plt.savefig(ruta_guardar, bbox_inches='tight')
    # devolver la imagen con todo pintada y una lista de los ángulos
    plt.show()
  return imagen, diferencias

## Código principal

In [14]:
def guardar_comparacion_csv(nombre, region, medida_original, medida_predicha, tipo_medida, ruta_salida):
  media = None
  ruta_csv = os.path.join(ruta_salida, f"comparacion_medidas_{tipo_medida}_{HOY}.csv")
  os.makedirs(ruta_salida, exist_ok=True)

  existe = os.path.isfile(ruta_csv)
  with open(ruta_csv, mode='a', newline='') as archivo_csv:
    writer = csv.writer(archivo_csv)
    if not existe:
      writer.writerow([
        "nombre_imagen",
        "region",
        "numero_vertebras_original",
        "numero_vertebras_predicha",
        "medidas_originales",
        "medidas_predichas",
        "media_diferencia_absoluta",
        "maxima_diferencia_absoluta",
        "desviacion_estandar_absoluta",
      ])

    num_comparaciones = min(len(medida_original), len(medida_predicha))
    if num_comparaciones > 0:
      diferencias = [abs(medida_predicha[i] - medida_original[i]) for i in range(num_comparaciones)]
      media = np.mean(diferencias)
      writer.writerow([
        nombre,
        region,
        len(medida_original),
        len(medida_predicha),
        medida_original[:num_comparaciones],
        medida_predicha[:num_comparaciones],
        round(media, 3),
        round(np.max(diferencias), 3),
        round(np.std(diferencias), 3)
      ])
      # Devolver también originales recortados para calcular R² y MPE
      originales_recortados = medida_original[:num_comparaciones]
      predichos_recortados = medida_predicha[:num_comparaciones]
      return media, diferencias, originales_recortados, predichos_recortados

  return media, [], [], []

In [15]:
def calcular_metricas(diferencias, originales):
  diferencias = np.array(diferencias)
  originales = np.array(originales)
  n = len(diferencias)
  if n == 0:
    return {}
  
  mae = np.mean(diferencias)
  mse = np.mean(diferencias**2)
  rmse = np.sqrt(mse)

  std_error = np.std(diferencias, ddof = 1) / np.sqrt(n)
  ic_95 = t.ppf(0.975, n-1) * std_error

  ss_res = np.sum(diferencias**2)
  ss_tot = np.sum((originales - np.mean(originales))**2)
  r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else float('nan')

  resultados = {
    'mae': round(mae, 4),
    'mse': round(mse, 4),
    'rmse': round(rmse, 4),
    'std_error': round(std_error, 4),
    'ic_95': round(ic_95, 4),
    'r2': round(r2, 4),
  }

  return resultados

In [16]:
def guardar_metricas_csv(metricas_dict, ruta_csv, tipo):
  os.makedirs(os.path.dirname(ruta_csv), exist_ok=True)
  claves = set()
  for val in metricas_dict.values():
    claves.update(val.keys())
  
  campos = ["tipo", "region"] + sorted(claves)

  with open(ruta_csv, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=campos)
    writer.writeheader()

    for region, metricas in metricas_dict.items():
      fila = {"tipo": tipo, "region": region}
      fila.update({k: metricas.get(k, "") for k in claves})
      writer.writerow(fila)

  print(f"Métricas guardadas en: {ruta_csv}")

In [27]:
def graficar_errores_por_region(diferencias_por_region, titulo, ruta_salida):
  os.makedirs(ruta_salida, exist_ok=True)

  regiones = list(diferencias_por_region.keys())
  datos = [diferencias_por_region[r] for r in regiones]

  todas_los_datos = np.concatenate(datos)
  bins = np.histogram_bin_edges(todas_los_datos, bins=30)

  # Histograma 
  plt.figure(figsize=(10, 6))
  for reg, vals in zip(regiones, datos):
    plt.hist(vals, bins=bins, alpha=0.6, label=reg.capitalize(), edgecolor='black')
  plt.title(f"Histograma - {titulo}")
  plt.xlabel("Error absoluto")
  plt.ylabel("Frecuencia")
  plt.legend()
  plt.tight_layout()
  plt.savefig(os.path.join(ruta_salida, f"histograma_{titulo.lower().replace(' ', '_')}.png"), dpi = 300)
  plt.close()

  # Boxplot
  plt.figure(figsize=(8, 6))
  plt.boxplot(datos, labels=[r.capitalize() for r in regiones], patch_artist=True,
              boxprops=dict(facecolor='lightblue'),
              medianprops=dict(color='red'))
  plt.title(f"Boxplot — {titulo}")
  plt.ylabel("Error absoluto")
  plt.tight_layout()
  plt.savefig(os.path.join(ruta_salida, f"boxplot_{titulo.lower().replace(' ', '_')}.png"), dpi=300)
  plt.close()

  print(f"Gráficas guardadas en: {ruta_salida}")

In [18]:
def mostrar_imagenes_comparadas(original, limpio, tipo, ruta_salida, nombre):
  ruta = os.path.join(ruta_salida, 'comparacion_metricas', tipo.lower())
  os.makedirs(ruta, exist_ok=True)
  plt.figure(figsize=(10, 10))
  plt.subplot(1, 2, 1)
  plt.title(f"Original - {tipo}")
  plt.imshow(original, cmap='gray')
  plt.axis('off')
  plt.subplot(1, 2, 2)
  plt.title(f"Segmentación - {tipo}")
  plt.imshow(limpio, cmap='gray')
  plt.axis('off')

  plt.savefig(os.path.join(ruta, nombre), bbox_inches='tight')
  plt.show()
  plt.close()

#### Sobre modelo general
1. Sobre el conjuntos NHANESS II

In [ ]:
ruta_entrada = os.path.join(ruta_procesado, con_mascara, mod_general)
ruta_salida = os.path.join(ruta_procesado, con_mascara, mod_general, metricas)
os.makedirs(ruta_salida, exist_ok=True)

diferencias_angulares = {'cervical': [], 'lumbar': []}
diferencias_traslacionales = {'cervical': [], 'lumbar': []}
originales_angulares = {'cervical': [], 'lumbar': []}
originales_traslacionales = {'cervical': [], 'lumbar': []}
# Calcular medidas de cada imagen
for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  # Cargar la imagen y la mascara
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_mascara_limpia = os.path.join(ruta_entrada, mascara_limpias, img_name)
  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara = cv2.imread(ruta_mascara, cv2.IMREAD_GRAYSCALE)
  mascara_limpia = cv2.imread(ruta_mascara_limpia, cv2.IMREAD_GRAYSCALE)
  # -- Medidas en la máscara limpia (segmentación) --
  # Calcular medidas angulares
  # se guarda en ruta_salida/angulares
  dif_img_limpia_angular, dif_angular_limpia = calcular_medidas_angulares(mascara_limpia, imagen, img_name, debug=False, tipo='general')
  ruta_imagen_angular = os.path.join(ruta_salida, angulares, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_angular), exist_ok=True)
  cv2.imwrite(ruta_imagen_angular, dif_img_limpia_angular)
  # Calcular medidas translacionales
  # se guarda en ruta_salida/translacionales
  dif_img_limpia_translacional, dif_translacional_limpia = calcular_medidas_translacionales(mascara_limpia, imagen, img_name, debug=False, tipo='general')
  ruta_imagen_translacional = os.path.join(ruta_salida, translacionales, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_translacional), exist_ok=True)
  cv2.imwrite(ruta_imagen_translacional, dif_img_limpia_translacional)
  # -- Medidas en la máscara original --
  # Calcular medidas angulares
  dif_img_ori_angular, dif_angular_ori= calcular_medidas_angulares(mascara, imagen, img_name, debug=False)
  mostrar_imagenes_comparadas(dif_img_ori_angular, dif_img_limpia_angular, "Angulares", ruta_salida, img_name)
  tipo = "cervical" if "cervical" in img_name.lower() else "lumbar"
  media_ang, difs_ang, ori_ang, pred_ang = guardar_comparacion_csv(
    nombre=img_name,
    region=tipo,
    medida_original=dif_angular_ori,
    medida_predicha=dif_angular_limpia,
    tipo_medida="angular",
    ruta_salida=ruta_salida
  )
  if media_ang is not None:
    diferencias_angulares[tipo].append(media_ang)
    originales_angulares[tipo].extend(ori_ang)

  # Calcular medidas translacionales
  dif_img_ori_translacional, dif_translacional_ori = calcular_medidas_translacionales(mascara, imagen, img_name, debug=False)
  mostrar_imagenes_comparadas(dif_img_ori_translacional, dif_img_limpia_translacional, "Translacionales", ruta_salida, img_name)
  media_tras, difs_tras, ori_tras, pred_tras = guardar_comparacion_csv(
    nombre=img_name,
    region=tipo,
    medida_original=dif_translacional_ori,
    medida_predicha=dif_translacional_limpia,
    tipo_medida="translacional",
    ruta_salida=ruta_salida
  )
  if media_tras is not None:
    diferencias_traslacionales[tipo].append(media_tras)
    originales_traslacionales[tipo].extend(ori_tras)

metricas_ang = {
  region: calcular_metricas(difs, originales=originales_angulares[region])
  for region, difs in diferencias_angulares.items()
}
guardar_metricas_csv(metricas_ang, os.path.join(ruta_salida, "resumen_metricas_angulares.csv"), "angular")
graficar_errores_por_region(diferencias_angulares, "Errores Angulares", ruta_salida)

metricas_trans = {
    region: calcular_metricas(difs, originales=originales_traslacionales[region])
    for region, difs in diferencias_traslacionales.items()
}
guardar_metricas_csv(metricas_trans, os.path.join(ruta_salida, "resumen_metricas_translacionales.csv"), "translacional")
graficar_errores_por_region(diferencias_traslacionales, "Errores Translacionales", ruta_salida)

2. Sobre el conjunto proporcionado por el médico (Julio)

In [ ]:
ruta_entrada = os.path.join(ruta_procesado, sin_mascara, mod_general)
ruta_salida = os.path.join(ruta_procesado, sin_mascara, mod_general, metricas)
os.makedirs(ruta_salida, exist_ok=True)

# Calcular medidas de cada imagen
for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  # Cargar la imagen y la mascara
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_mascara_limpia = os.path.join(ruta_entrada, mascara_limpias, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara_limpia = cv2.imread(ruta_mascara_limpia, cv2.IMREAD_GRAYSCALE)

  # Calcular medidas angulares
  # se guarda en ruta_salida/angulares
  dif_imagen, diff_angular = calcular_medidas_angulares(mascara_limpia, imagen, img_name, debug=False, tipo='general')
  # guardar imagen con medidas angulares
  ruta_imagen_angular = os.path.join(ruta_salida, angulares, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_angular), exist_ok=True)
  cv2.imwrite(ruta_imagen_angular, dif_imagen)

  plt.figure(figsize=(10, 10))
  plt.title("Angulares - Cervical")
  plt.imshow(dif_imagen, cmap='gray')
  plt.axis('off')
  plt.show()

  # Calcular medidas translacionales
  # se guarda en ruta_salida/translacionales
  dif_imagen, diff_translacional = calcular_medidas_translacionales(mascara_limpia, imagen, img_name, debug=False, tipo='general')
  ruta_imagen_translacional = os.path.join(ruta_salida, translacionales, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_translacional), exist_ok=True)
  cv2.imwrite(ruta_imagen_translacional, dif_imagen)
  calcular_medidas_translacionales(mascara_limpia, imagen, img_name)
  plt.figure(figsize=(10, 10))
  plt.title("Translacionales - Cervical")
  plt.imshow(dif_imagen, cmap='gray')
  plt.axis('off')
  plt.show()


#### Sobre el modelo cervical
1. Sobre el conjunto de NHANES II

In [ ]:
ruta_entrada = os.path.join(ruta_procesado, con_mascara, mod_cervical)
ruta_salida = os.path.join(ruta_procesado, con_mascara, mod_cervical, metricas)
os.makedirs(ruta_salida, exist_ok=True)

diferencias_angulares = {'cervical': []}
diferencias_traslacionales = {'cervical': []}
originales_angulares = {'cervical': []}
originales_traslacionales = {'cervical': []}
# Calcular medidas de cada imagen
for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  # Cargar la imagen y la mascara
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_mascara_limpia = os.path.join(ruta_entrada, mascara_limpias, img_name)
  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara = cv2.imread(ruta_mascara, cv2.IMREAD_GRAYSCALE)
  mascara_limpia = cv2.imread(ruta_mascara_limpia, cv2.IMREAD_GRAYSCALE)
  # -- Medidas en la máscara limpia (segmentación) --
  # Calcular medidas angulares
  # se guarda en ruta_salida/angulares
  dif_img_limpia_angular, dif_angular_limpia = calcular_medidas_angulares(mascara_limpia, imagen, img_name, debug=False, tipo='cervical')
  ruta_imagen_angular = os.path.join(ruta_salida, angulares, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_angular), exist_ok=True)
  cv2.imwrite(ruta_imagen_angular, dif_img_limpia_angular)
  # Calcular medidas translacionales
  # se guarda en ruta_salida/translacionales
  dif_img_limpia_translacional, dif_translacional_limpia = calcular_medidas_translacionales(mascara_limpia, imagen, img_name, debug=False, tipo='cervical')
  ruta_imagen_translacional = os.path.join(ruta_salida, translacionales, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_translacional), exist_ok=True)
  cv2.imwrite(ruta_imagen_translacional, dif_img_limpia_translacional)
  # -- Medidas en la máscara original --
  # Calcular medidas angulares
  dif_img_ori_angular, dif_angular_ori= calcular_medidas_angulares(mascara, imagen, img_name, debug=False)
  mostrar_imagenes_comparadas(dif_img_ori_angular, dif_img_limpia_angular, "Angulares", ruta_salida, img_name)
  tipo = "cervical"
  media_ang, difs_ang, ori_ang, pred_ang = guardar_comparacion_csv(
    nombre=img_name,
    region=tipo,
    medida_original=dif_angular_ori,
    medida_predicha=dif_angular_limpia,
    tipo_medida="angular",
    ruta_salida=ruta_salida
  )
  if media_ang is not None:
    diferencias_angulares[tipo].append(media_ang)
    originales_angulares[tipo].extend(ori_ang)

  # Calcular medidas translacionales
  dif_img_ori_translacional, dif_translacional_ori = calcular_medidas_translacionales(mascara, imagen, img_name, debug=False)
  mostrar_imagenes_comparadas(dif_img_ori_translacional, dif_img_limpia_translacional, "Translacionales", ruta_salida, img_name)
  media_tras, difs_tras, ori_tras, pred_tras = guardar_comparacion_csv(
    nombre=img_name,
    region=tipo,
    medida_original=dif_translacional_ori,
    medida_predicha=dif_translacional_limpia,
    tipo_medida="translacional",
    ruta_salida=ruta_salida
  )
  if media_tras is not None:
    diferencias_traslacionales[tipo].append(media_tras)
    originales_traslacionales[tipo].extend(ori_tras)

metricas_ang = {
  region: calcular_metricas(difs, originales=originales_angulares[region])
  for region, difs in diferencias_angulares.items()
}
guardar_metricas_csv(metricas_ang, os.path.join(ruta_salida, "resumen_metricas_angulares.csv"), "angular")
graficar_errores_por_region(diferencias_angulares, "Errores Angulares", ruta_salida)

metricas_trans = {
    region: calcular_metricas(difs, originales=originales_traslacionales[region])
    for region, difs in diferencias_traslacionales.items()
}
guardar_metricas_csv(metricas_trans, os.path.join(ruta_salida, "resumen_metricas_translacionales.csv"), "translacional")
graficar_errores_por_region(diferencias_traslacionales, "Errores Translacionales", ruta_salida)

2. Sobre el conjunto proporcionado por el médico

In [ ]:
ruta_entrada = os.path.join(ruta_procesado, sin_mascara, mod_cervical)
ruta_salida = os.path.join(ruta_procesado, sin_mascara, mod_cervical, metricas)
os.makedirs(ruta_salida, exist_ok=True)

# Calcular medidas de cada imagen
for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  # Cargar la imagen y la mascara
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_mascara_limpia = os.path.join(ruta_entrada, mascara_limpias, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara_limpia = cv2.imread(ruta_mascara_limpia, cv2.IMREAD_GRAYSCALE)

  # Calcular medidas angulares
  # se guarda en ruta_salida/angulares
  dif_imagen, diff_angular = calcular_medidas_angulares(mascara_limpia, imagen, img_name, debug=False, tipo='cervical')
  # guardar imagen con medidas angulares
  ruta_imagen_angular = os.path.join(ruta_salida, angulares, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_angular), exist_ok=True)
  cv2.imwrite(ruta_imagen_angular, dif_imagen)

  plt.figure(figsize=(10, 10))
  plt.title("Angulares - Cervical")
  plt.imshow(dif_imagen, cmap='gray')
  plt.axis('off')
  plt.show()

  # Calcular medidas translacionales
  # se guarda en ruta_salida/translacionales
  dif_imagen, diff_translacional = calcular_medidas_translacionales(mascara_limpia, imagen, img_name, debug=False, tipo='cervical')
  ruta_imagen_translacional = os.path.join(ruta_salida, translacionales, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_translacional), exist_ok=True)
  cv2.imwrite(ruta_imagen_translacional, dif_imagen)
  calcular_medidas_translacionales(mascara_limpia, imagen, img_name)
  plt.figure(figsize=(10, 10))
  plt.title("Translacionales - Cervical")
  plt.imshow(dif_imagen, cmap='gray') 
  plt.axis('off')
  plt.show()


#### Sobre el modelo lumbar
1. Sobre el conjunto de NHANES II

In [ ]:
ruta_entrada = os.path.join(ruta_procesado, con_mascara, mod_lumbar)
ruta_salida = os.path.join(ruta_procesado, con_mascara, mod_lumbar, metricas)
os.makedirs(ruta_salida, exist_ok=True)

diferencias_angulares = {'lumbar': []}
diferencias_traslacionales = {'lumbar': []}
originales_angulares = {'lumbar': []}
originales_traslacionales = {'lumbar': []}
# Calcular medidas de cada imagen
for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  # Cargar la imagen y la mascara
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_mascara_limpia = os.path.join(ruta_entrada, mascara_limpias, img_name)
  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara = cv2.imread(ruta_mascara, cv2.IMREAD_GRAYSCALE)
  mascara_limpia = cv2.imread(ruta_mascara_limpia, cv2.IMREAD_GRAYSCALE)
  # -- Medidas en la máscara limpia (segmentación) --
  # Calcular medidas angulares
  # se guarda en ruta_salida/angulares
  dif_img_limpia_angular, dif_angular_limpia = calcular_medidas_angulares(mascara_limpia, imagen, img_name, debug=False, tipo='lumbar')
  ruta_imagen_angular = os.path.join(ruta_salida, angulares, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_angular), exist_ok=True)
  cv2.imwrite(ruta_imagen_angular, dif_img_limpia_angular)
  # Calcular medidas translacionales
  # se guarda en ruta_salida/translacionales
  dif_img_limpia_translacional, dif_translacional_limpia = calcular_medidas_translacionales(mascara_limpia, imagen, img_name, debug=False, tipo='lumbar')
  ruta_imagen_translacional = os.path.join(ruta_salida, translacionales, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_translacional), exist_ok=True)
  cv2.imwrite(ruta_imagen_translacional, dif_img_limpia_translacional)
  # -- Medidas en la máscara original --
  # Calcular medidas angulares
  dif_img_ori_angular, dif_angular_ori= calcular_medidas_angulares(mascara, imagen, img_name, debug=False)
  mostrar_imagenes_comparadas(dif_img_ori_angular, dif_img_limpia_angular, "Angulares", ruta_salida, img_name)
  tipo = "lumbar"
  media_ang, difs_ang, ori_ang, pred_ang = guardar_comparacion_csv(
    nombre=img_name,
    region=tipo,
    medida_original=dif_angular_ori,
    medida_predicha=dif_angular_limpia,
    tipo_medida="angular",
    ruta_salida=ruta_salida
  )
  if media_ang is not None:
    diferencias_angulares[tipo].append(media_ang)
    originales_angulares[tipo].extend(ori_ang)

  # Calcular medidas translacionales
  dif_img_ori_translacional, dif_translacional_ori = calcular_medidas_translacionales(mascara, imagen, img_name, debug=False)
  mostrar_imagenes_comparadas(dif_img_ori_translacional, dif_img_limpia_translacional, "Translacionales", ruta_salida, img_name)
  media_tras, difs_tras, ori_tras, pred_tras = guardar_comparacion_csv(
    nombre=img_name,
    region=tipo,
    medida_original=dif_translacional_ori,
    medida_predicha=dif_translacional_limpia,
    tipo_medida="translacional",
    ruta_salida=ruta_salida
  )
  if media_tras is not None:
    diferencias_traslacionales[tipo].append(media_tras)
    originales_traslacionales[tipo].extend(ori_tras)

metricas_ang = {
  region: calcular_metricas(difs, originales=originales_angulares[region])
  for region, difs in diferencias_angulares.items()
}
guardar_metricas_csv(metricas_ang, os.path.join(ruta_salida, "resumen_metricas_angulares.csv"), "angular")
graficar_errores_por_region(diferencias_angulares, "Errores Angulares", ruta_salida)

metricas_trans = {
    region: calcular_metricas(difs, originales=originales_traslacionales[region])
    for region, difs in diferencias_traslacionales.items()
}
guardar_metricas_csv(metricas_trans, os.path.join(ruta_salida, "resumen_metricas_translacionales.csv"), "translacional")
graficar_errores_por_region(diferencias_traslacionales, "Errores Translacionales", ruta_salida)

2. Sobre el conjunto proporcionado por el médico

In [ ]:
ruta_entrada = os.path.join(ruta_procesado, sin_mascara, mod_lumbar)
ruta_salida = os.path.join(ruta_procesado, sin_mascara, mod_lumbar, metricas)
os.makedirs(ruta_salida, exist_ok=True)

# Calcular medidas de cada imagen
for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  # Cargar la imagen y la mascara
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_mascara_limpia = os.path.join(ruta_entrada, mascara_limpias, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara_limpia = cv2.imread(ruta_mascara_limpia, cv2.IMREAD_GRAYSCALE)

  # Calcular medidas angulares
  # se guarda en ruta_salida/angulares
  dif_imagen, diff_angular = calcular_medidas_angulares(mascara_limpia, imagen, img_name, debug=False, tipo='lumbar')
  # guardar imagen con medidas angulares
  ruta_imagen_angular = os.path.join(ruta_salida, angulares, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_angular), exist_ok=True)
  cv2.imwrite(ruta_imagen_angular, dif_imagen)

  plt.figure(figsize=(10, 10))
  plt.title("Angulares - Lumbar")
  plt.imshow(dif_imagen, cmap='gray')
  plt.axis('off')
  plt.show()  

  # Calcular medidas translacionales
  # se guarda en ruta_salida/translacionales
  dif_imagen, diff_translacional = calcular_medidas_translacionales(mascara_limpia, imagen, img_name, debug=False, tipo='lumbar')
  ruta_imagen_translacional = os.path.join(ruta_salida, translacionales, img_name)
  os.makedirs(os.path.dirname(ruta_imagen_translacional), exist_ok=True)
  cv2.imwrite(ruta_imagen_translacional, dif_imagen)
  calcular_medidas_translacionales(mascara_limpia, imagen, img_name)

  plt.figure(figsize=(10, 10))
  plt.title("Translacionales - Lumbar")
  plt.imshow(dif_imagen, cmap='gray')
  plt.axis('off')
  plt.show()
